<a href="https://colab.research.google.com/github/odarasimi/RealEstatePriceModel/blob/master/Utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import json
from flask import Flask, request, jsonify, render_template
import numpy as np

In [ ]:
from google.colab import files
pick = files.upload()

In [ ]:
from google.colab import files
columns = files.upload()

*utility*

In [4]:
__locations = None
__model = None
__data_columns = None
 
def load_saved_data():
  global __locations
  global __data_columns 
  global __model
 
  with open("columns.json", "r") as f:
    __data_columns = json.load(f)["data_columns"]
    __locations = __data_columns[3:]
  
  if __model is None:
    with open("banglore_home_prices_model.pickle","rb") as p:
      __model = pickle.load(p)
 
def get_location_names():
  return __locations
 
def get_data_columns():
  return __data_columns
 
def get_estimated_price(location,total_sqft,bath,sizes):
  try:
      loc = __data_columns.index(location.lower())
  except:
      loc = -1
 
  x = np.zeros(len(__data_columns))
  x[0] = total_sqft
  x[1] = bath
  x[2] = sizes
  if loc>=0:
    x[loc] = 1
 
  return round(__model.predict([x])[0],2)

*testing the functions*

In [11]:
(load_saved_data())
print (get_location_names())
print (get_data_columns())
print (get_estimated_price('2nd phase judicial layout',1000,1, 1))

['1st block jayanagar', '1st phase jp nagar', '2nd phase judicial layout', '2nd stage nagarbhavi', '5th block hbr layout', '5th phase jp nagar', '6th phase jp nagar', '7th phase jp nagar', '8th phase jp nagar', '9th phase jp nagar', 'aecs layout', 'abbigere', 'akshaya nagar', 'ambalipura', 'ambedkar nagar', 'amruthahalli', 'anandapura', 'ananth nagar', 'anekal', 'anjanapura', 'ardendale', 'arekere', 'attibele', 'beml layout', 'btm 2nd stage', 'btm layout', 'babusapalaya', 'badavala nagar', 'balagere', 'banashankari', 'banashankari stage ii', 'banashankari stage iii', 'banashankari stage v', 'banashankari stage vi', 'banaswadi', 'banjara layout', 'bannerghatta', 'bannerghatta road', 'basavangudi', 'basaveshwara nagar', 'battarahalli', 'begur', 'begur road', 'bellandur', 'benson town', 'bharathi nagar', 'bhoganhalli', 'billekahalli', 'binny pete', 'bisuvanahalli', 'bommanahalli', 'bommasandra', 'bommasandra industrial area', 'bommenahalli', 'brookefield', 'budigere', 'cv raman nagar', 'c

**APP**

In [6]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://ne5lf8gglg-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [39]:
app = Flask(__name__,template_folder="drive/My Drive/Colab Notebooks")

In [40]:
@app.route("/get_location_names", methods=["GET"])
def get_the_location_names():
  return jsonify({"locations": get_location_names()})

In [41]:
@app.route("/predict", methods=["POST"])
def predict():
  total_sqft = float(request.form["total_sqft"])
  location = request.form["location"]
  bath = int(request.form["bath"])
  sizes = int(request.form["sizes"])
  return jsonify({"estimated_price": "{} rupees".format(get_estimated_price(location,total_sqft,bath,sizes))})

In [42]:
@app.route("/")
def index():
  return render_template("account.html")

In [ ]:
if __name__ == "__main__":
  load_saved_data()
  print ("app running...")
  app.run()